In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [14]:

from tqdm import tqdm
import re
import pickle
import json
from langchain.embeddings import SentenceTransformerEmbeddings
from transformers import AutoModel, AutoTokenizer
from langchain.vectorstores import Chroma
import json
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AdamW
import numpy as np
import random
from IPython.display import HTML, display
import getpass
import logging
import os
import yaml
from datasets import load_dataset
import numpy as np; np.random.seed(123)
import pandas as pd
from tqdm import tqdm
from IPython.display import HTML, display
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import json
import pandas as pd

In [3]:
os.getpid()

1158173

In [42]:
gt= pd.read_csv('Categorization dataset.csv')
gt.head()
len(gt)

319

In [44]:
dataset_c = []
for _, (Material_Name, Caption, DOI, Descriptor, Category) in gt.iterrows():
    Prompt = f"""How do you categorize the material called '{Material_Name}' based on the following information: 
    Caption of the table that described the chemical composition of this material: {Caption}
    Descriptor from another language model: {Descriptor}
    
    Category of the material '{Material_Name}': """

    dataset_c.append({"Prompt": Prompt, "Winner": str(Category)})
    
len(dataset_c)

319

In [45]:
gt.loc[1]

Material Name                                                   FA
Caption          Major elements present in the S/S agents and M...
DOI                                  10.1016/j.jhazmat.2010.08.127
Descriptor        Class F fly ash, specifically low calcium fly...
Category          Ash, Fly, Class F, Byproduct of coal combustion 
Name: 1, dtype: object

In [46]:
fine_tuning_dataset=[]

In [47]:
# Reformatting for fine-tuning
fine_tuning_dataset = []
for item in dataset_c:
    fine_tuning_item = {
        "messages": [
            {"role": "system", "content": "You are an assistant trained to classify the materials based on the description provided and your own knowledge about materials. "},
            {"role": "user", "content": json.dumps(item["Prompt"])},
            {"role": "assistant", "content": json.dumps(item["Winner"])}
        ]
    }
    fine_tuning_dataset.append(fine_tuning_item)
    


In [48]:
len(fine_tuning_dataset)

319

In [49]:
fine_tuning_dataset[1]

{'messages': [{'role': 'system',
   'content': 'You are an assistant trained to classify the materials based on the description provided and your own knowledge about materials. Consider these keywords and abbreviations that are used in concrete industry to classify the materials: {cheat_sheet}'},
  {'role': 'user',
   'content': '"How do you categorize the material called \'FA\' based on the following information: \\n    Caption of the table that described the chemical composition of this material: Major elements present in the S/S agents and MSWI waste.\\n    Descriptor from another language model:  Class F fly ash, specifically low calcium fly ash from coal combustion, is used as the main aluminosilicate agent and geopolymer precursor in this study.\\n    \\n    Category of the material \'FA\': "'},
  {'role': 'assistant',
   'content': '"Ash, Fly, Class F, Byproduct of coal combustion "'}]}

In [50]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(fine_tuning_dataset, test_size=0.1)

In [51]:

with open('fine_tuning_TE_ENR.jsonl', 'w') as file:
    for item in fine_tuning_dataset:
        json.dump(item, file)
        file.write('\n')
        

with open('fine_tuning_TRAIN_TE_ENR.jsonl', 'w') as file:
    for item in train_data:
        json.dump(item, file)
        file.write('\n')
        

with open('fine_tuning_TEST_TE_ENR.jsonl', 'w') as file:
    for item in test_data:
        json.dump(item, file)
        file.write('\n')

In [53]:
len(train_data)

293

In [54]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer api_key" \
  -F "purpose=fine-tune" \
  -F "file=@fine_tuning_TRAIN_TE_ENR.jsonl" 

{
  "object": "file",
  "id": "file-n57msb21C1844BMySxGKKeNL",
  "purpose": "fine-tune",
  "filename": "fine_tuning_TRAIN_TE_ENR.jsonl",
  "bytes": 281170,
  "created_at": 1706208998,
  "status": "processed",
  "status_details": null
}


In [55]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer api_key" \
  -F "purpose=fine-tune" \
  -F "file=@fine_tuning_TEST_TE_ENR.jsonl" 

{
  "object": "file",
  "id": "file-6jvYmnyirwy4K9Rm6Ay9SZ0U",
  "purpose": "fine-tune",
  "filename": "fine_tuning_TEST_TE_ENR.jsonl",
  "bytes": 24570,
  "created_at": 1706209054,
  "status": "processed",
  "status_details": null
}


In [56]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

client.fine_tuning.jobs.create(
  training_file="file-n57msb21C1844BMySxGKKeNL", 
  validation_file="file-6jvYmnyirwy4K9Rm6Ay9SZ0U", 
  model="gpt-3.5-turbo-1106", 
  hyperparameters={
    "n_epochs":"auto"
  }
)

FineTuningJob(id='ftjob-QPd8qCJe8ailytd99bJY7EdU', created_at=1706209099, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-CGIKMf8W0RLVflPNXCgE6SzT', result_files=[], status='validating_files', trained_tokens=None, training_file='file-n57msb21C1844BMySxGKKeNL', validation_file='file-6jvYmnyirwy4K9Rm6Ay9SZ0U')

In [83]:
def save_responses(responses):
    # Implement the logic to save 'responses'
    # For example, saving to a file:
    filename = f'responses_some.csv'
    # Convert responses to DataFrame and save as CSV
    pd.DataFrame(responses, columns=['Material Name','Caption','DOI','Descriptor','Response']).to_csv(filename, index=False)


In [58]:

from openai import OpenAI
client = OpenAI(api_key=api_key)



In [60]:


# Load the data from CSV files
materials_df = pd.read_csv('final_compositions.csv', encoding='cp1252')


materials_df.head()

,Material Name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,LOI,Caption,DOI,Descriptor
0,Zeolitic rock from Sakhaptinskoje deposit,64.8,12.77,2.46,NaN,1.84,NaN,0.35,NaN,NaN,NaN,11.12,Chemical composition of primary material.,10.1016/j.conbuildmat.2009.10.010,The Zeolitic rock from Sakhaptinskoe deposit ...
1,Ash microspheres,62.3,23.40,4.00,NaN,4.21,NaN,1.3,NaN,NaN,NaN,1.74,Chemical composition of primary material.,10.1016/j.conbuildmat.2009.10.010,Ash microspheres are spherical particles prim...
2,Vermiculite,34.0,12.00,6.00,NaN,13.00,NaN,NaN,NaN,4.00,NaN,NaN,Chemical composition of primary material.,10.1016/j.conbuildmat.2009.10.010,Vermiculite is a mineral composed primarily o...
3,HAC,0.2,71.00,0.10,28.5,0.40,NaN,NaN,NaN,NaN,0.2,0.5,Oxide composition and chemical properties of H...,10.1016/j.conbuildmat.2010.01.022,The material named 'HAC' contains approximate...
4,OPC,22.1,5.00,3.00,63.8,1.60,2,NaN,NaN,0.64,0.35,1.2,Oxide composition and chemical properties of H...,10.1016/j.conbuildmat.2010.01.022,The material named 'OPC' has an oxide composi...


In [84]:
responses = []

In [85]:

for i, (index, row) in enumerate(tqdm(materials_df.iterrows(), total=materials_df.shape[0])):
    RR =0
    material_name = row['Material Name']
    doi = row['DOI']
    caption = row['Caption']
    descriptor = row['Descriptor']
    try: 
        
        Prompt = f"""How do you categorize the material called '{material_name}' based on the following information: 
        Caption of the table that described the chemical composition of this material: {caption}
        Descriptor from another language model: {descriptor}

        Category of the material '{material_name}': """

        # Check if this caption and DOI already exist in the existing DataFrame
        response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:personal::8kzjZwlt",messages=[
        {"role": "system", "content": "You are an assistant trained to classify the materials based on the description provided and your own knowledge about materials."},
        {"role": "user", "content": Prompt}],
        temperature=0.0,
        max_tokens=100,
        top_p=1,)

        RR=response.choices[0].message.content

        # Check if it's time to save the responses
        if (i + 1) % 100 == 0:
            save_responses(responses)


    except Exception as e:
        print(e)
        
    RR=response.choices[0].message.content
    responses.append((material_name, caption, doi,descriptor, RR))

 55%|█████▌    | 8016/14462 [40:24<23:31,  4.57it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 14462/14462 [1:10:38<00:00,  3.41it/s]


In [86]:
save_responses(responses)

In [103]:
import pandas as pd

# Load data
data = pd.read_csv('all_materials_extracted_fgpt.csv', encoding='unicode_escape')
oxide_columns = ['SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'SO3', 'TiO2', 'MnO', 'K2O', 'Na2O', 'LOI']

        
# Function to sum oxide values and handle errors
def sum_oxides(row):
    try:
        return row[oxide_columns].astype(float).sum()
    except ValueError:
        return 'error'

# Apply the function to each row
data['Sum'] = data.apply(sum_oxides, axis=1)

# Identify rows with errors
rows_with_errors = data['Sum'] == 'error'

# Extract rows for manual check
data_non_numeric = data[rows_with_errors]

# Save these rows to a new file for manual checking
data_non_numeric.to_excel('manual_check.xlsx', index=False)

# Optionally, remove these rows from the original DataFrame
data = data[~rows_with_errors]
data.to_csv('Final_ext_compositions.csv', index=False)
